In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
nickname = []
rating = []
title=[]
voice=[]
author=[]

In [3]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [4]:
driver = webdriver.Chrome('C:/Users/USER/Desktop/데잇걸즈/파이썬/[어수웅] 파이썬 프로그래밍/chromedriver')
driver.implicitly_wait(3)
driver.get("http://m.podbbang.com/audiobooks?category_id=105") #메인페이지
scroll(driver, 0.6) #스크롤 끝까지 내려
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [5]:
for i in range(1,len(soup.select('ul.audiobook-list-component__list>li.audiobook-list-item-component>h3.title'))):
    element = driver.find_element_by_xpath(f'//*[@id="audiobookIndex"]/div[2]/div/ul/li[{i}]')
    driver.execute_script("arguments[0].click();", element)
    scroll(driver, 0.6) #스크롤 끝까지 내려
    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser') #특정 책 soup
    n = int(soup2.select('div.audiobook-reviews-component>h3.title>span')[0].text.split(' ')[1].strip('명')) #n정의
    for i in range((n-3)//12+1):
        더보기 = driver.find_element_by_xpath('//*[@id="audiobookDetails"]/div[5]/div/button[1]')
        더보기.click()
    time.sleep(1)
    html = driver.page_source
    soup3 = BeautifulSoup(html, 'html.parser') #댓글 다 나와있는 soup
    for i in range(n):
        c=soup3.select('dt.title')[0].text
        a=soup3.select('h3.nickname')[i].text
        b=soup3.select('div.star_rating>span')[i].get('style').split(' ')[1][:-2]
        d=soup3.select('dl.audiobook-details__info>dd')[1].text[3:]
        e=soup3.select('dl.audiobook-details__info>dd')[2].text[2:]
        nickname.append(a)
        rating.append(b)
        title.append(c)
        voice.append(d)
        author.append(e)
    driver.back() # 책 목록으로 돌아가
    scroll(driver, 0.6) #스크롤 끝까지 내려

In [6]:
booklist = pd.DataFrame({
    'title': title,
    'nickname':nickname,
    'rating':rating,
    'voice': voice,
    'author': author
    
})

booklist

,title,nickname,rating,voice,author
0,[강연형] 제국대학의 조센징,히히덕,100,정종현,정종현
1,[강연형] 제국대학의 조센징,스키드로,100,정종현,정종현
2,[강연형] 제국대학의 조센징,니체짱,100,정종현,정종현
3,[강연형] 제국대학의 조센징,pimono,90,정종현,정종현
4,[강연형] 제국대학의 조센징,뚱스키,90,정종현,정종현
...,...,...,...,...,...
580,전쟁과 평화(요약본),호이호이슈슈,100,김은정,톨스토이
581,전쟁과 평화(요약본),쌀까마,100,김은정,톨스토이
582,전쟁과 평화(요약본),히든인천러브,100,김은정,톨스토이
583,전쟁과 평화(요약본),아산둔포황소야,100,김은정,톨스토이


In [7]:
booklist.groupby('title').count()

,nickname,rating,voice,author
title,,,,
[강연형] 제국대학의 조센징,498,498,498,498
전쟁과 평화(요약본),87,87,87,87


In [8]:
booklist.to_csv('C:/Users/USER/Desktop/데잇걸즈/데모데이/역사.csv', encoding='cp949')